# INSTRUCTIONS

Workflow: Run each cell in order. Put or extract data into relevant folders as defined in section 2. Make remaining folders in drive.  **Cells denoted with * may require extra action.**


```
Folder Structure Suggested for Section 2:
drive/
    train_data/ <- input midis go here
    train_output/
        graphs/ <- train/val accuracy plots go here
        intermed/ <- intermediate weights, preprocessing go here
        stats/ (currently not being used)
        midi/ <- output midis go here
```




# 1 Imports

In [2]:
# Import Data Manip, Debug
import glob
import pickle
import numpy as np
import pandas as pd
import pdb
import matplotlib.pyplot as plt

# Import Music21
!pip install music21
from music21 import converter, instrument, note, chord, stream

# Import Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import TimeDistributed
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import CuDNNLSTM
from keras.layers import CuDNNGRU
from keras.layers import Activation
from keras.layers import Bidirectional
from keras.layers import RepeatVector
from keras.layers import Flatten
from keras.regularizers import L1L2
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras.callbacks import History

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [3]:
# stuff needed for colaboratory to connect with drive
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
··········


# 2 Mount and Set Directories*

In [4]:
# mount drive
!mkdir -p drive
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [0]:
# data specific
data_type = 'rock'
var = 'melody_'

EDM_CORPUS = '/edm_'
ROCK_CORPUS = '/rock_'
FOLK_CORPUS = '/folk_'

if data_type == "rock":
  CORPUS = ROCK_CORPUS
elif data_type == "edm":
  CORPUS = EDM_CORPUS
elif data_type == "folk":
  CORPUS = FOLK_CORPUS
  
NAME = CORPUS + var


DATA_FOLDER = 'drive/EE239AS/Project/train_data' + NAME[:-1]
GRAPHS_FOLDER = 'drive/EE239AS/Project/train_output/graphs'
MIDI_OUTPUT_FOLDER = 'drive/EE239AS/Project/train_output/midi'
INTERMED_FOLDER = 'drive/EE239AS/Project/train_output/intermed'
STATS_FOLDER = 'drive/EE239AS/Project/train_output/stats'





# 3 Set Training Parameters*

In [0]:
# SET PARAMETERS (ACTION)
RESTS = False
DURATION_BATCH_SIZE = 256
NOTE_BATCH_SIZE = 128
# SPECIFY PARAMETERS TO TEST AS LIST
DROPOUTS = [0, 0.5, 0.3]
# DROPOUTS = [0.5]
MODEL_SIZES = [64, 256, 512]
EPOCHS = 10

# 4 Preprocess Data (MIDI Data Into Notes Corpus and Duration Corpus)

In [0]:
# Test

notes_corpus = []
durations_corpus = []

for file in glob.glob(DATA_FOLDER + "/*.mid"):
    try:
      print("Extracting MIDI File: ", file)
      midi_stream = converter.parse(file)

      notes = None

      partition = instrument.partitionByInstrument(midi_stream)

      if not RESTS:
        # No rests
        if partition: 
            notes = partition.parts[0].recurse()
        else: 
            notes = midi_stream.flat.notes
      else:
        # With rests
        if partition: 
            notes = partition.parts[0].recurse()
        else: 
            notes = midi_stream.notesAndRests
          
      in_song_notes = []
      in_song_durations = []
      for element in notes:
          in_song_durations.append(element.duration.quarterLength)
          if isinstance(element, note.Note):
              in_song_notes.append(str(element.pitch))
          elif RESTS and isinstance(element, note.Rest):
              in_song_notes.append("R")
          elif isinstance(element, chord.Chord):
              in_song_notes.append(element.root().nameWithOctave)
#               in_song.append('.'.join(str(n) for n in element.normalOrder))
    except:
      pass
    
    notes_corpus.append(in_song_notes)
    durations_corpus.append(in_song_durations)
            
# Write
with open(INTERMED_FOLDER + NAME + 'notes_corpus', 'wb+') as filepath:
    pickle.dump(notes_corpus, filepath)
    
with open(INTERMED_FOLDER + NAME + 'durations_corpus', 'wb+') as filepath:
    pickle.dump(durations_corpus, filepath)

Extracting MIDI File:  drive/EE239AS/Project/train_data/rock_melody/3001.mid
Extracting MIDI File:  drive/EE239AS/Project/train_data/rock_melody/2937.mid
Extracting MIDI File:  drive/EE239AS/Project/train_data/rock_melody/458.mid
Extracting MIDI File:  drive/EE239AS/Project/train_data/rock_melody/3040.mid
Extracting MIDI File:  drive/EE239AS/Project/train_data/rock_melody/4449.mid
Extracting MIDI File:  drive/EE239AS/Project/train_data/rock_melody/14997.mid
Extracting MIDI File:  drive/EE239AS/Project/train_data/rock_melody/3012.mid
Extracting MIDI File:  drive/EE239AS/Project/train_data/rock_melody/4464.mid
Extracting MIDI File:  drive/EE239AS/Project/train_data/rock_melody/14998.mid
Extracting MIDI File:  drive/EE239AS/Project/train_data/rock_melody/15231.mid
Extracting MIDI File:  drive/EE239AS/Project/train_data/rock_melody/15255.mid
Extracting MIDI File:  drive/EE239AS/Project/train_data/rock_melody/15227.mid
Extracting MIDI File:  drive/EE239AS/Project/train_data/rock_melody/1582

# 5 Preprocess Corpus Into Train Data

## 5.1 Notes Corpus

In [7]:
# load notes corpus
with open(INTERMED_FOLDER + NAME + 'notes_corpus', 'rb') as filepath:
    notes_corpus = pickle.load(filepath)
    
# If doing learning for one song at a time only
flattened_notes_corpus = []
for song_notes in notes_corpus:
    flattened_notes_corpus += song_notes

vocab_size = len(set(flattened_notes_corpus))
print(vocab_size)

# Produce input output sequences
window_size = 60
note_sequence_input = []
next_note_output = []

notes = sorted(set(flattened_notes_corpus))
note2int = dict((note, num) for num, note in enumerate(notes))

for i in range(0, len(notes_corpus)):
    for j in range(0, len(notes_corpus[i]) - window_size):
        current_sequence = [note2int[note] for note in notes_corpus[i][j:window_size+j]]
        next_note = note2int[notes_corpus[i][window_size+j]]
        note_sequence_input.append(current_sequence)
        next_note_output.append(next_note)

98


In [8]:
# check train and label shapes
training_data = np.reshape(note_sequence_input, (len(note_sequence_input), window_size , 1))
training_data = training_data / float(vocab_size)
print('Train shape: ' + str(training_data.shape))
training_label = np_utils.to_categorical(next_note_output, num_classes=vocab_size)
print('Label shape: ' + str(training_label.shape))

Train shape: (373606, 60, 1)
Label shape: (373606, 98)


## 5.2 Durations Corpus

In [9]:
# load notes corpus
with open(INTERMED_FOLDER + NAME + 'durations_corpus', 'rb') as filepath:
    duration_corpus = pickle.load(filepath)

# Learn on one song at a time
flattened_duration_corpus = []
for song_durations in duration_corpus:
    flattened_duration_corpus += song_durations
    
import collections
counter = collections.Counter(flattened_duration_corpus)
print(counter)


duration_vocab_size = len(set(flattened_duration_corpus))
print(duration_vocab_size)

# Produce input output sequences
duration_window_size = 30
duration_sequence_input = []
next_duration_output = []

durations = sorted(set(flattened_duration_corpus))
duration2int = dict((duration, num) for num, duration in enumerate(durations))

print(duration2int)
# Write
# with open(INTERMED_FOLDER + "/edm_duration_counter", 'wb+') as filepath:
#     pickle.dump(counter, filepath)

for i in range(0, len(duration_corpus)):
    for j in range(0, len(duration_corpus[i]) - duration_window_size):
        current_duration_sequence = [duration2int[note] for note in duration_corpus[i][j:duration_window_size+j]]
        next_duration = duration2int[duration_corpus[i][duration_window_size+j]]
        duration_sequence_input.append(current_duration_sequence)
        next_duration_output.append(next_duration)

Counter({0.5: 149809, 0.25: 123537, 1.0: 53619, Fraction(1, 3): 22153, 0.75: 12868, 2.0: 12855, 1.5: 11551, Fraction(2, 3): 8093, 0.0: 7133, 4.0: 4559, Fraction(1, 6): 3596, 1.25: 2960, 2.5: 2667, 3.0: 2597, 8.0: 1799, 1.75: 1685, Fraction(4, 3): 1644, 3.5: 1056, 2.25: 791, Fraction(5, 3): 684, 3.75: 662, 5.0: 651, 4.5: 628, 2.75: 625, 6.0: 496, 3.25: 450, Fraction(7, 3): 387, 16.0: 364, 7.0: 344, Fraction(8, 3): 321, Fraction(1, 12): 256, 6.5: 251, 7.5: 242, 4.25: 220, Fraction(10, 3): 211, 12.0: 208, 5.75: 207, 5.5: 206, 7.75: 192, Fraction(11, 3): 161, 4.75: 159, 32.0: 151, 8.5: 129, 6.25: 127, Fraction(5, 12): 122, 9.0: 116, 5.25: 113, 10.0: 112, Fraction(13, 3): 97, 12.5: 90, 24.0: 89, 7.25: 79, 11.0: 76, 15.5: 75, 6.75: 70, Fraction(7, 6): 70, 14.5: 65, 8.25: 64, 10.5: 58, 15.0: 56, 9.5: 56, 13.0: 55, 14.0: 53, 20.0: 48, 16.5: 48, Fraction(7, 12): 48, 13.5: 45, 11.5: 44, Fraction(23, 3): 44, Fraction(20, 3): 44, 64.0: 43, 8.75: 42, Fraction(19, 3): 41, 12.25: 40, 28.0: 39, 18.0: 

In [10]:
# check train and label shapes
duration_training_data = np.reshape(duration_sequence_input, (len(duration_sequence_input), duration_window_size , 1))
duration_training_data = duration_training_data / float(duration_vocab_size)
print('Train shape: ' + str(duration_training_data.shape))
duration_training_label = np_utils.to_categorical(next_duration_output, num_classes=duration_vocab_size)
print('Label shape: ' + str(duration_training_label.shape))

Train shape: (419742, 30, 1)
Label shape: (419742, 2353)


In [0]:
print(len(duration_corpus[1]))
print(len(next_duration_output))

# 6 Train

In [0]:
from keras import backend as K
from keras.engine.topology import Layer
#from keras import initializations
from keras import initializers, regularizers, constraints


class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Example:
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
        """
        self.supports_masking = True
        #self.init = initializations.get('glorot_uniform')
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        # eij = K.dot(x, self.W) TF backend doesn't support it

        # features_dim = self.W.shape[0]
        # step_dim = x._keras_shape[1]

        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
    #print weigthted_input.shape
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        #return input_shape[0], input_shape[-1]
        return input_shape[0],  self.features_dim

In [0]:
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import BatchNormalization

def create_model(network_input, n_vocab, model_size, dropout):
  model = Sequential()
  reg = L1L2(0, 0)
  model.add(LSTM(
      model_size,
      input_shape=(network_input.shape[1], network_input.shape[2]),
      return_sequences=True,
      dropout=dropout, recurrent_dropout=0.3
  ))
  model.add(CuDNNLSTM(model_size, return_sequences=True, kernel_regularizer=reg))
  model.add(Dropout(dropout))
  model.add(CuDNNLSTM(model_size, kernel_regularizer=reg))
  model.add(Dense(128))
  model.add(Dropout(dropout))
  model.add(Dense(n_vocab))
  model.add(Activation('softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

def create_attention_model(network_input, n_vocab, model_size, dropout, window_size):
  model = Sequential()
  reg = L1L2(0, 0)  
  #LSTM
  model.add(LSTM(
      model_size,
      input_shape=(network_input.shape[1], network_input.shape[2]),
      return_sequences=True,
      dropout=dropout, recurrent_dropout=0.3
  ))
  model.add(CuDNNLSTM(model_size, return_sequences=True, kernel_regularizer=reg))
  model.add(Dropout(dropout))
  model.add(CuDNNLSTM(model_size, return_sequences=True, kernel_regularizer=reg))
  model.add(Attention(window_size))
  model.add(Dense(128))
  model.add(Dropout(dropout))
  model.add(Dense(n_vocab))
  model.add(Activation('softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model


def create_attention_callback_list(model_size, dropout, model_type):
  model_size += "-attention"
  filepath = INTERMED_FOLDER + '%skarpathy-model-weights-%s-%s-%s.hdf5' % (NAME, model_type, model_size, dropout)
  model_checkpoint = ModelCheckpoint(
    filepath,
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
  )
  return [model_checkpoint], filepath

def create_callback_list(model_size, dropout, model_type):
  filepath = INTERMED_FOLDER + '%skarpathy-model-weights-%s-%s-%s.hdf5' % (NAME, model_type, model_size, dropout)
  model_checkpoint = ModelCheckpoint(
    filepath,
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
  )
  return [model_checkpoint], filepath

# acc history
def setup_plot(dropout, size):
  plt.title('Model Accuracy vs. Epoc with Dropout=%s Size=%s' % (dropout, size))
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  
def plot_history(history, model_type, dropout, size):
  plt.plot(history.history['acc'], label="%s train accuracy" % model_type)
  plt.plot(history.history['val_acc'], label="%s val accuracy" % model_type)

def save_plot(file_path):
  plt.legend()
  plt.savefig(file_path)
  plt.clf()
  
def predict_duration(mode, model, WEIGHT_PATH):
  # Prediction
  model.load_weights(WEIGHT_PATH)
  starting_sequence = np.random.randint(219, size=duration_window_size)
  pattern_sequence = starting_sequence.tolist()
  prediction_output = []

  duration2note = dict((num, note) for num, note in enumerate(durations))
  print (duration2note)
  
  for i in range(400):
      prediction_input = np.reshape(pattern_sequence, (1, len(pattern_sequence), 1))
      prediction_input = prediction_input / float(duration_vocab_size)

      if i%3 == 0:   
          print('\r', 'Predicting.  Duration: ', i, end='')  
      if i%3 == 1:   
          print('\r', 'Predicting.. Duration: ', i, end='')
      if i%3 == 2:   
          print('\r', 'Predicting...Duration: ', i, end='')
      prediction = model.predict(prediction_input, verbose=0)
      
      # Predict based on prob dist      
      if mode == "pred":      
        prediction_values = np.arange(len(prediction[0]))
        prediction_prob = np.asarray(list(prediction[0])) / float(sum(prediction[0]))
        index = np.random.choice(prediction_values, 1, p=prediction_prob)
        note_instance = duration2note[int(index[0])]
        prediction_output.append(note_instance)
        
      # Most probable note prediction
      elif mode == "max":
        index = np.argmax(prediction)
        note_instance = duration2note[index]
        prediction_output.append(note_instance)


      pattern_sequence.append(index)
      pattern_sequence = pattern_sequence[1:len(pattern_sequence)]    

  prediction_output = prediction_output[300:len(prediction_output)]
#   prediction_output = prediction_output[0:60]

  # Write
  with open(INTERMED_FOLDER + ("%sduration_prediction_output" % NAME), 'wb+') as filepath:
      pickle.dump(prediction_output, filepath)
  
  return prediction_output

def predict_note(mode, model, WEIGHT_PATH):
  model.load_weights(WEIGHT_PATH)
  # Prediction
  starting_sequence = np.random.randint(219, size=window_size)
  pattern_sequence = starting_sequence.tolist()
  prediction_output = []

  int2note = dict((num, note) for num, note in enumerate(notes))
  print (int2note)

  for i in range(400):
      prediction_input = np.reshape(pattern_sequence, (1, len(pattern_sequence), 1))
      prediction_input = prediction_input / float(vocab_size)

      if i%3 == 0:   
          print('\r', 'Predicting.  Note: ', i, end='')  
      if i%3 == 1:   
          print('\r', 'Predicting.. Note: ', i, end='')
      if i%3 == 2:   
          print('\r', 'Predicting...Note: ', i, end='')
      prediction = model.predict(prediction_input, verbose=0)

      # Predict based on prob dist      
      if mode == "pred":
#         import pdb; pdb.set_trace()
        prediction_values = np.arange(len(prediction[0]))
        prediction_prob = np.asarray(list(prediction[0])) / float(sum(prediction[0]))
        index = np.random.choice(prediction_values, 1, p=prediction_prob)
        note_instance = int2note[int(index[0])]
        prediction_output.append(note_instance)

      # Most probable note prediction
      elif mode == "max":
        index = np.argmax(prediction)
        note_instance = int2note[index]
        prediction_output.append(note_instance)



      pattern_sequence.append(index)
      pattern_sequence = pattern_sequence[1:len(pattern_sequence)]    

  prediction_output = prediction_output[300:len(prediction_output)]
#   prediction_output = prediction_output[0:60]
  # Write
  with open(INTERMED_FOLDER + ("%snotes_prediction_output" % NAME), 'wb+') as filepath:
      pickle.dump(prediction_output, filepath)
  return prediction_output

def output_midi(prediction_output, duration_prediction_output, dropout, model_size, mode):
  offset = 0
  output_notes = []
  for pattern, duration in zip(prediction_output, duration_prediction_output):

      if ('.' in pattern) or pattern.isdigit():
          chord_array = pattern.split('.')
          chord_notes = []
          for note_instance in chord_array:
              note_object = note.Note(int(note_instance))
              note_object.duration.quarterLength = duration
              note_object.storedInstrument = instrument.Piano()
              chord_notes.append(note_object)
          chord_object = chord.Chord(chord_notes)
          chord_object.offset = offset
          output_notes.append(chord_object)
      elif 'R' == pattern:
          note_object = note.Rest()
          note_object.duration.quarterLength = duration
          note_object.offset = offset
          output_notes.append(note_object)
      else:
          note_object = note.Note(pattern)
          note_object.duration.quarterLength = duration
          note_object.offset = offset
          note_object.storedInstrument = instrument.Piano()
          output_notes.append(note_object)

      offset += 0.5

  midi_stream = stream.Stream(output_notes)
  midi_stream.write('midi', fp=MIDI_OUTPUT_FOLDER + NAME + '%s_%s_%s.mid' % (dropout, model_size, mode))
  print('\nWrote midi...')

In [15]:
notes_histories = {}
duration_histories = {}
dropout = 0
model_size = 64
mode = "pred"


setup_plot(dropout, model_size)
print('Running duration training on dropout:%s size:%s' % (dropout, model_size))
duration_callbacks, duration_weight_path = create_attention_callback_list('duration', dropout, model_size)
duration_model = create_attention_model(duration_training_data, duration_training_label.shape[1], model_size, dropout, 30)
duration_histories[(dropout, model_size)] = duration_model.fit(duration_training_data, duration_training_label, epochs=EPOCHS, batch_size=DURATION_BATCH_SIZE, callbacks=duration_callbacks, validation_split=0.2)
plot_history(duration_histories[(dropout, model_size)], 'Durations', dropout, model_size)
# output intermed duration
duration_prediction = predict_duration(mode, duration_model, duration_weight_path)

print('\n\nRunning notes training on dropout:%s size:%s' % (dropout, model_size))
notes_callbacks, note_weight_path = create_attention_callback_list('notes', dropout, model_size)
notes_model = create_attention_model(training_data, training_label.shape[1], model_size, dropout, 60)
notes_histories[(dropout, model_size)] = notes_model.fit(training_data, training_label, epochs=EPOCHS, batch_size=NOTE_BATCH_SIZE, callbacks=notes_callbacks, validation_split=0.2)
plot_history(notes_histories[(dropout, model_size)], 'Notes', dropout, model_size)
save_plot(GRAPHS_FOLDER + NAME + 'dropout=%s_size=%s.jpg' % (dropout, model_size))
# output intermed notes
note_prediction = predict_note(mode, notes_model, note_weight_path)

# output final midi
output_midi(note_prediction, duration_prediction, dropout, model_size, mode + "attention")

Running duration training on dropout:0 size:64
Instructions for updating:
Use tf.cast instead.
Train on 335793 samples, validate on 83949 samples
Epoch 1/10
335793/335793 [==============================] - 144s 429us/step - loss: 2.2085 - acc: 0.3393 - val_loss: 2.1506 - val_acc: 0.3367
Epoch 2/10
335793/335793 [==============================] - 137s 409us/step - loss: 2.1238 - acc: 0.3431 - val_loss: 2.1493 - val_acc: 0.3367
Epoch 3/10
335793/335793 [==============================] - 138s 410us/step - loss: 2.0334 - acc: 0.3987 - val_loss: 1.9045 - val_acc: 0.4933
Epoch 4/10
335793/335793 [==============================] - 137s 409us/step - loss: 1.8176 - acc: 0.4936 - val_loss: 1.7981 - val_acc: 0.5162
Epoch 5/10
335793/335793 [==============================] - 137s 407us/step - loss: 1.7248 - acc: 0.5188 - val_loss: 1.7373 - val_acc: 0.5346
Epoch 6/10
335793/335793 [==============================] - 135s 402us/step - loss: 1.6608 - acc: 0.5303 - val_loss: 1.6776 - val_acc: 0.5439
Ep

<Figure size 576x396 with 0 Axes>

In [15]:
notes_histories = {}
duration_histories = {}
dropout = 0
model_size = 64
mode = "pred"


setup_plot(dropout, model_size)
print('Running duration training on dropout:%s size:%s' % (dropout, model_size))
duration_callbacks, duration_weight_path = create_callback_list('duration', dropout, model_size)
duration_model = create_model(duration_training_data, duration_training_label.shape[1], model_size, dropout)
duration_histories[(dropout, model_size)] = duration_model.fit(duration_training_data, duration_training_label, epochs=EPOCHS, batch_size=DURATION_BATCH_SIZE, callbacks=duration_callbacks, validation_split=0.2)
plot_history(duration_histories[(dropout, model_size)], 'Durations', dropout, model_size)
# output intermed duration
duration_prediction = predict_duration(mode, duration_model, duration_weight_path)

print('\n\nRunning notes training on dropout:%s size:%s' % (dropout, model_size))
notes_callbacks, note_weight_path = create_callback_list('notes', dropout, model_size)
notes_model = create_model(training_data, training_label.shape[1], model_size, dropout)
notes_histories[(dropout, model_size)] = notes_model.fit(training_data, training_label, epochs=EPOCHS, batch_size=NOTE_BATCH_SIZE, callbacks=notes_callbacks, validation_split=0.2)
plot_history(notes_histories[(dropout, model_size)], 'Notes', dropout, model_size)
save_plot(GRAPHS_FOLDER + NAME + 'dropout=%s_size=%s.jpg' % (dropout, model_size))
# output intermed notes
note_prediction = predict_note(mode, notes_model, note_weight_path)

# output final midi
output_midi(note_prediction, duration_prediction, dropout, model_size, mode + "attention")

Running duration training on dropout:0 size:64
Instructions for updating:
Use tf.cast instead.
Train on 335793 samples, validate on 83949 samples
Epoch 1/10
335793/335793 [==============================] - 132s 392us/step - loss: 2.1989 - acc: 0.3419 - val_loss: 2.1482 - val_acc: 0.3367
Epoch 2/10
335793/335793 [==============================] - 131s 391us/step - loss: 2.1229 - acc: 0.3428 - val_loss: 2.1374 - val_acc: 0.3367
Epoch 3/10
335793/335793 [==============================] - 129s 384us/step - loss: 1.9820 - acc: 0.4297 - val_loss: 1.8835 - val_acc: 0.5073
Epoch 4/10
335793/335793 [==============================] - 128s 380us/step - loss: 1.8107 - acc: 0.4956 - val_loss: 1.7857 - val_acc: 0.5179
Epoch 5/10
335793/335793 [==============================] - 128s 380us/step - loss: 1.7208 - acc: 0.5174 - val_loss: 1.7566 - val_acc: 0.5288
Epoch 6/10
335793/335793 [==============================] - 127s 377us/step - loss: 1.6597 - acc: 0.5331 - val_loss: 1.7038 - val_acc: 0.5398
Ep

<Figure size 576x396 with 0 Axes>

In [14]:
d = 0
ms = 64

mode = "pred"

duration_callbacks, duration_weight_path = create_callback_list('duration', d, ms)
notes_callbacks, note_weight_path = create_callback_list('notes', d, ms)

duration_model = create_model(duration_training_data, duration_vocab_size, ms, d)
notes_model = create_model(training_data, vocab_size, ms, d)

print(note_weight_path)
print(duration_weight_path)

pn = predict_note(mode, notes_model, note_weight_path)
print(pn)

dn = predict_duration(mode, duration_model, duration_weight_path)
print(dn)

output_midi(pn, dn, d, ms, mode)

drive/EE239AS/Project/train_output/intermed/rock_melody_karpathy-model-weights-64-notes-0.hdf5
drive/EE239AS/Project/train_output/intermed/rock_melody_karpathy-model-weights-64-duration-0.hdf5
{0: 'A0', 1: 'A1', 2: 'A2', 3: 'A3', 4: 'A4', 5: 'A5', 6: 'A6', 7: 'A7', 8: 'B--1', 9: 'B-0', 10: 'B-1', 11: 'B-2', 12: 'B-3', 13: 'B-4', 14: 'B-5', 15: 'B-6', 16: 'B-7', 17: 'B0', 18: 'B1', 19: 'B2', 20: 'B3', 21: 'B4', 22: 'B5', 23: 'B6', 24: 'B7', 25: 'C#-1', 26: 'C#1', 27: 'C#2', 28: 'C#3', 29: 'C#4', 30: 'C#5', 31: 'C#6', 32: 'C#7', 33: 'C#8', 34: 'C-1', 35: 'C1', 36: 'C2', 37: 'C3', 38: 'C4', 39: 'C5', 40: 'C6', 41: 'C7', 42: 'C8', 43: 'D1', 44: 'D2', 45: 'D3', 46: 'D4', 47: 'D5', 48: 'D6', 49: 'D7', 50: 'D8', 51: 'E--1', 52: 'E-0', 53: 'E-1', 54: 'E-2', 55: 'E-3', 56: 'E-4', 57: 'E-5', 58: 'E-6', 59: 'E-7', 60: 'E1', 61: 'E2', 62: 'E3', 63: 'E4', 64: 'E5', 65: 'E6', 66: 'E7', 67: 'F#1', 68: 'F#2', 69: 'F#3', 70: 'F#4', 71: 'F#5', 72: 'F#6', 73: 'F#7', 74: 'F0', 75: 'F1', 76: 'F2', 77: 'F3'

In [25]:
d = 0
ms = 64

mode = "pred"

duration_callbacks, duration_weight_path = create_attention_callback_list('duration', d, ms)
notes_callbacks, note_weight_path = create_attention_callback_list('notes', d, ms)

duration_model = create_attention_model(duration_training_data, duration_vocab_size, ms, d, 30)
notes_model = create_attention_model(training_data, vocab_size, ms, d, 60)

print(note_weight_path)
print(duration_weight_path)

pn = predict_note(mode, notes_model, note_weight_path)
print(pn)

dn = predict_duration(mode, duration_model, duration_weight_path)
print(dn)

output_midi(pn, dn, d, ms, mode)

drive/EE239AS/Project/train_output/intermed/rock_melody_karpathy-model-weights-64-notes-attention-0.hdf5
drive/EE239AS/Project/train_output/intermed/rock_melody_karpathy-model-weights-64-duration-attention-0.hdf5
{0: 'A0', 1: 'A1', 2: 'A2', 3: 'A3', 4: 'A4', 5: 'A5', 6: 'A6', 7: 'A7', 8: 'B--1', 9: 'B-0', 10: 'B-1', 11: 'B-2', 12: 'B-3', 13: 'B-4', 14: 'B-5', 15: 'B-6', 16: 'B-7', 17: 'B0', 18: 'B1', 19: 'B2', 20: 'B3', 21: 'B4', 22: 'B5', 23: 'B6', 24: 'B7', 25: 'C#-1', 26: 'C#1', 27: 'C#2', 28: 'C#3', 29: 'C#4', 30: 'C#5', 31: 'C#6', 32: 'C#7', 33: 'C#8', 34: 'C-1', 35: 'C1', 36: 'C2', 37: 'C3', 38: 'C4', 39: 'C5', 40: 'C6', 41: 'C7', 42: 'C8', 43: 'D1', 44: 'D2', 45: 'D3', 46: 'D4', 47: 'D5', 48: 'D6', 49: 'D7', 50: 'D8', 51: 'E--1', 52: 'E-0', 53: 'E-1', 54: 'E-2', 55: 'E-3', 56: 'E-4', 57: 'E-5', 58: 'E-6', 59: 'E-7', 60: 'E1', 61: 'E2', 62: 'E3', 63: 'E4', 64: 'E5', 65: 'E6', 66: 'E7', 67: 'F#1', 68: 'F#2', 69: 'F#3', 70: 'F#4', 71: 'F#5', 72: 'F#6', 73: 'F#7', 74: 'F0', 75: 'F1'

In [18]:
 for d in DROPOUTS:
  for ms in MODEL_SIZES:
    mode = "pred"

    duration_callbacks, duration_weight_path = create_callback_list('duration', d, ms)
    notes_callbacks, note_weight_path = create_callback_list('notes', d, ms)

    duration_model = create_model(duration_training_data, duration_vocab_size, ms, d)
    notes_model = create_model(training_data, vocab_size, ms, d)

    print(note_weight_path)
    print(duration_weight_path)

    pn = predict_note(mode, notes_model, note_weight_path)
    print(pn)

    dn = predict_duration(mode, duration_model, duration_weight_path)
    print(dn)

    output_midi(pn, dn, d, ms, mode)

TypeError: ignored

## 6.1 Helpers to Create Model

## 6.2 Hyperparameter Optimization

In [0]:
notes_histories = {}
duration_histories = {}
for dropout in DROPOUTS:
  for model_size in MODEL_SIZES:
    setup_plot(dropout, model_size)
    print('Running duration training on dropout:%s size:%s' % (dropout, model_size))
    duration_callbacks, duration_weight_path = create_callback_list('duration', dropout, model_size)
    duration_model = create_model(duration_training_data, duration_vocab_size, model_size, dropout)
    duration_histories[(dropout, model_size)] = duration_model.fit(duration_training_data, duration_training_label, epochs=EPOCHS, batch_size=DURATION_BATCH_SIZE, callbacks=duration_callbacks, validation_split=0.2)
    plot_history(duration_histories[(dropout, model_size)], 'Durations', dropout, model_size)
    # output intermed duration
    duration_prediction = predict_duration(duration_model, duration_weight_path)
    
    print('\n\nRunning notes training on dropout:%s size:%s' % (dropout, model_size))
    notes_callbacks, note_weight_path = create_callback_list('notes', dropout, model_size)
    notes_model = create_model(training_data, vocab_size, model_size, dropout)
    notes_histories[(dropout, model_size)] = notes_model.fit(training_data, training_label, epochs=EPOCHS, batch_size=NOTE_BATCH_SIZE, callbacks=notes_callbacks, validation_split=0.2)
    plot_history(notes_histories[(dropout, model_size)], 'Notes', dropout, model_size)
    save_plot(GRAPHS_FOLDER + NAME + 'dropout=%s_size=%s.jpg' % (dropout, model_size))
    # output intermed notes
    note_prediction = predict_note(notes_model, note_weight_path)
    
    # output final midi
    output_midi(note_prediction, duration_prediction, dropout, model_size)

Running duration training on dropout:0.5 size:64
Train on 335793 samples, validate on 83949 samples
Epoch 1/10
335793/335793 [==============================] - 144s 427us/step - loss: 2.2189 - acc: 0.3393 - val_loss: 2.1489 - val_acc: 0.3367
Epoch 2/10
335793/335793 [==============================] - 138s 411us/step - loss: 2.1356 - acc: 0.3432 - val_loss: 2.1402 - val_acc: 0.3367
Epoch 3/10
335793/335793 [==============================] - 138s 412us/step - loss: 2.1255 - acc: 0.3429 - val_loss: 2.1398 - val_acc: 0.3367
Epoch 4/10
335793/335793 [==============================] - 140s 416us/step - loss: 2.1124 - acc: 0.3454 - val_loss: 2.1650 - val_acc: 0.3000
Epoch 5/10
335793/335793 [==============================] - 138s 410us/step - loss: 2.0864 - acc: 0.3769 - val_loss: 2.3192 - val_acc: 0.2690
Epoch 6/10
335793/335793 [==============================] - 138s 412us/step - loss: 2.0676 - acc: 0.3938 - val_loss: 2.2326 - val_acc: 0.2949
Epoch 7/10
335793/335793 [======================